In [1]:
!pip install nltk
!pip install xgboost
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Load your data (replace with your actual data loading)
# Assuming your data is in a CSV file named 'data.csv'
import pandas as pd
new_columns=pd.read_csv('../data/neiss_10p_sample_new_columns.csv')
data_10P = pd.read_csv('../data/neiss_10p_sample.csv')

# sematic=pd.read_csv('../data/10P_sematic_bert.csv')
embedding=pd.read_csv('../data/gist_embedding_10p_v0.csv')

#embedding_2=pd.read_csv('../xwang3306/aliba_embedding_10p.csv')

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


[nltk_data] Downloading package punkt to /home/eric/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/eric/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/eric/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eric/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data=data_10P.merge(new_columns,how='inner',on='CPSC_Case_Number').merge(embedding,how='inner',on='CPSC_Case_Number').reset_index(drop=True)
data.head(5)

,Unnamed: 0,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Hispanic,Body_Part,Diagnosis,...,374,375,376,377,378,379,380,381,382,383
0,0,221032332,2022-09-24,14,1,0,0,0.0,34,71,...,0.034040,-0.036666,0.008797,0.024984,-0.048995,0.002708,-0.020119,-0.058474,-0.022191,0.038710
1,1,181109464,2018-10-30,28,1,1,0,0.0,79,71,...,0.024239,-0.000142,0.007269,-0.015545,-0.040856,-0.042023,-0.050380,0.008760,-0.002266,0.043031
2,2,210103105,2020-10-24,35,1,0,0,0.0,30,53,...,0.060739,0.014632,0.022998,-0.055498,-0.040177,0.024382,-0.086975,-0.045614,-0.022894,-0.013169
3,3,161157997,2016-11-15,214,2,0,0,0.0,76,53,...,0.027231,-0.016303,-0.013848,-0.046165,-0.034185,0.006101,-0.041465,-0.041030,0.040551,0.013185
4,4,181107411,2018-10-21,4,1,0,0,0.0,92,72,...,0.019272,-0.061428,0.017961,0.035712,0.021475,0.038897,-0.028660,0.035182,-0.031062,-0.006973


In [3]:
data.columns

Index(['Unnamed: 0', 'CPSC_Case_Number', 'Treatment_Date', 'Age', 'Sex',
       'Race', 'Other_Race', 'Hispanic', 'Body_Part', 'Diagnosis',
       ...
       '374', '375', '376', '377', '378', '379', '380', '381', '382', '383'],
      dtype='object', length=416)

In [4]:
import numpy as np
data['Disposition_recode']=np.nan
data.loc[((data['Disposition']==1)), 'Disposition_recode'] = 0
data.loc[((data['Disposition']==2)), 'Disposition_recode'] = 1
data.loc[((data['Disposition']==4)), 'Disposition_recode'] = 2
data.loc[((data['Disposition']==5)), 'Disposition_recode'] = 3
data.loc[((data['Disposition']==8)), 'Disposition_recode'] = 4
data=data[data['Disposition_recode'].notna()]

data['Disposition_recode_2']=0
data.loc[((data['Disposition_recode']>0)), 'Disposition_recode_2'] = 1

In [5]:
data=data[(data['Body_Part']!=0) & (data['Body_Part']!=84) & (data['Body_Part']!=85) & (data['Body_Part']!=86) & (data['Body_Part']!=87)]
data['Disposition_recode'].value_counts()
data['Disposition_recode_2'].value_counts() 

Disposition_recode_2
0    296227
1     32874
Name: count, dtype: int64

In [6]:
bdpt_dict={}
bdpt_dict[0]='INTERNAL'
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[78]='UPPERTRUNK(OLD)'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[84]='25-50% OF BODY'
bdpt_dict[85]='ALLPARTSBODY'
bdpt_dict[86]='OTHER(OLD)'
bdpt_dict[87]='NOTSTATED/UNK'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[90]='LOWERARM(OLD)'
bdpt_dict[91]='LOWERLEG(OLD)'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'


In [7]:
data['body_string']=data['Body_Part'].map(bdpt_dict)
data['Narrative_LLM']=data["activity_at_injury"].astype(str) + ' '+data["injury_mechanism"].astype(str)+ ' ' + data["object_involved"].astype(str)

In [8]:
# data['Narrative']=data['Narrative'].str.upper()
data[['Narrative']]

,Narrative
0,14YOM REPORTS HE FELL 1 WEEK AND COMPLAINS OF ...
1,A 28YOM BENT TO PICK UP CRATE AT HOME TO ED WI...
2,35YOMRIDING ON MOUNTAIN BIKE PRACTICING FELL D...
3,14 MONTH OLD FEMALE ABRASION FOR NOSE AND FORE...
4,4YR M PLAYING WITH TOY KITCHEN APPLIANCE AND ...
...,...
352046,11 YOM HELPING LOAD BICYCLE IN VAN CUT HEAD DX...
352047,47YOF PUSHING DOWN FOOD IN BLENDER MAKING A SM...
352048,2 YO MALE FELL DOWN STEPS DX HEAD INJURY
352049,38YOM WAS RIDING A BIKE AND FELL WRIST INJURY


In [9]:
replace_list=['ANKLE', 'ARM', 'BODY_PART', 'CHEST', 'CONTUSION', 'CUT', 'EAR', 'ELBOW', 'EYE', 'FACE', 'FINGER', 'FOOT', 'FOREHEAD', 'FRACTURE', 'FX', 'HAND', 'HEAD', 'HIP', 'KNEE', 'LAC', 'LACERATION', 'LEG', 'LOC', 'LOSE', 'NECK', 'PAIN', 'SHOULDER', 
'SPRAIN', 'STRAIN', 'SWELL', 'THUMB', 'TOE', 'WRIST','ABRASION', 'ACHE', 'BREAK', 'BURN', 'CHIN', 'CUT', 'ER', 'FRACTURE', 'FX', 'HIT', 'INJURY', 'LACERATION', 'LIP', 'LOSE', 'LOC', 'MOUTH', 'NOSE', 'PAIN', 'RIB', 'SCALP', 'SPRAIN', 'STRAIN', 'SWELL', 'TOE', 'TWIST', 'WRIST']

for i in replace_list:
  data['Narrative'] = data['Narrative'].str.replace(i, '')
  data['Narrative_LLM']= data['Narrative_LLM'].str.replace(i, '')


data['Narrative'] = data['Narrative'].str.replace('YOM', '')
data['Narrative'] = data['Narrative'].str.replace('YOF', '')
data['Narrative'] = data['Narrative'].str.replace('YR', '')
data['Narrative'] = data['Narrative'].str.replace('OLD', '')
data['Narrative'] = data['Narrative'].str.replace('MALE', '')
data['Narrative'] = data['Narrative'].str.replace('FEMALE', '')
data['Narrative'] = data['Narrative'].str.replace(' YO ', '')
data['Narrative'] = data['Narrative'].str.replace('YO ', '')
data['Narrative'] = data['Narrative'].str.replace(' F ', '')
data['Narrative'] = data['Narrative'].str.replace('YF', '')
data['Narrative'] = data['Narrative'].str.replace(' M ', '')
data['Narrative'] = data['Narrative'].str.replace('ACCIDENTALLY','')
data['Narrative'] = data['Narrative'].str.replace('AGO', '')
data['Narrative'] = data['Narrative'].str.replace('TODAY', '')
data['Narrative'] = data['Narrative'].str.replace('YESTERDAY', '')
data['Narrative'] = data['Narrative'].str.replace('PATIENT', '')
data['Narrative'] = data['Narrative'].str.replace(' PT ', '')
data['Narrative'] = data['Narrative'].str.replace('INJURY', '')
data['Narrative'] = data['Narrative'].str.replace('REPORT', '')
data['Narrative'] = data['Narrative'].str.replace('HURT', '')
data['Narrative'] = data['Narrative'].str.replace('INJ', '')
data['Narrative'] = data['Narrative'].str.replace('FELL', 'FALL')
data['Narrative'] = data['Narrative'].str.replace('INJURE', '')
data['Narrative'] = data['Narrative'].str.replace('JURED', '')
data['Narrative'] = data['Narrative'].str.replace('URED', '')
data['Narrative'] = data['Narrative'].str.replace(' ED', '')
data['Narrative'] = data['Narrative'].str.replace(' RT ', '')
data['Narrative'] = data['Narrative'].str.replace(' LT ', '')

data['Narrative_LLM'] = data['Narrative_LLM'].str.replace('fell', 'fall')


In [11]:
data_core=data.copy() #[core_col]
data_core.shape

(329101, 420)

In [12]:
# please change these hard coded numbers accordingly
data_core=data_core.sample(frac=1,random_state=42).reset_index(drop=True)
data_test=data_core.head(21000).reset_index(drop=True)
data_fit=data_core.tail(191347).reset_index(drop=True)
print(data_test.shape)
print(data_fit.shape)

(21000, 420)
(191347, 420)


In [15]:
data_ready = data_fit
data_ready_test = data_test

print(data_ready.shape)
print(data_ready_test.shape)

(191347, 420)
(21000, 420)


In [16]:
drop_list=['Narrative','Unnamed: 0',
 'CPSC_Case_Number',
 'Treatment_Date','Race',
 'Other_Race',
 'Hispanic','Diagnosis',
 'Other_Diagnosis',
 'Body_Part_2',
 'Diagnosis_2',
 'Other_Diagnosis_2',
 'Disposition','Fire_Involvement','Product_2',
 'Product_3',
 'Alcohol',
 'Drug','Stratum',
 'PSU',
 'Weight',
 'Year',
 'Month',
 'Day',
 'activity_at_injury',
 'injury_mechanism',
 'object_involved',
 # '#_prod',
 'Disposition_recode',
 'Disposition_recode_2',
 'body_string',
 'Narrative_LLM',
 # 'Processed_Narrative',
 # 'Processed_Narrative_LLM'
          ]

drop_list_test=['Narrative','Unnamed: 0',
 'CPSC_Case_Number',
 'Treatment_Date','Race',
 'Other_Race',
 'Hispanic','Diagnosis',
 'Other_Diagnosis',
 'Body_Part_2',
 'Diagnosis_2',
 'Other_Diagnosis_2',
 'Disposition','Fire_Involvement','Product_2',
 'Product_3',
 'Alcohol',
 'Drug','Stratum',
 'PSU',
 'Weight',
 'Year',
 'Month',
 'Day',
 'activity_at_injury',
 'injury_mechanism',
 'object_involved',
 # '#_prod',
 'Disposition_recode',
 'Disposition_recode_2',
 'body_string',
 # 'Narrative_LLM'
]
 

In [17]:
X = data_ready.drop(drop_list, axis=1) 
y = data_ready['Disposition_recode_2']

print(list(X.columns))

['Age', 'Sex', 'Body_Part', 'Location', 'Product_1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', 

In [18]:
X.head(10)

,Age,Sex,Body_Part,Location,Product_1,0,1,2,3,4,...,374,375,376,377,378,379,380,381,382,383
0,63,1,92,0,895,-0.007688,0.018969,0.069697,-0.006078,-0.011042,...,0.006437,-0.029620,0.023056,-0.054788,-0.013622,0.007888,0.018202,-0.000004,-0.012063,0.060675
1,20,2,79,0,3299,-0.067750,-0.017916,0.013089,0.018385,-0.021431,...,0.043872,0.007420,-0.024572,0.024614,-0.038932,0.033884,-0.049016,-0.091405,0.001964,0.015661
2,24,1,75,0,3286,-0.030343,-0.020277,-0.012162,0.023924,0.008585,...,0.034677,-0.007227,-0.027022,-0.008214,-0.046270,-0.022725,-0.052537,-0.008046,0.003771,0.022771
3,19,1,83,9,3235,-0.043329,-0.044473,0.037445,-0.108835,0.030722,...,-0.006613,0.009686,-0.026982,0.030318,-0.010904,0.021937,-0.015105,0.027934,0.004723,0.040201
4,11,1,33,0,3217,-0.090960,0.032737,0.021915,-0.021867,-0.014595,...,0.044465,-0.037258,-0.036151,-0.018563,-0.026852,0.042868,-0.113982,-0.024364,-0.046709,0.022000
5,2,1,76,1,4057,-0.044233,0.009734,0.057457,0.040132,0.009833,...,-0.030926,-0.008574,0.033612,-0.030632,0.029084,-0.027360,-0.027081,-0.004111,-0.027521,0.024481
6,9,1,36,0,1217,-0.048464,0.004734,0.039974,-0.031085,-0.000993,...,-0.028866,-0.010878,-0.008052,0.007033,-0.028309,0.046614,-0.086943,-0.016555,-0.102299,0.048339
7,38,2,32,5,3297,-0.031701,-0.000055,0.019885,-0.006134,-0.022459,...,0.055065,-0.008867,0.009221,-0.042574,-0.022461,0.000051,0.005730,0.003090,-0.045116,0.087371
8,25,1,93,1,1842,-0.031501,-0.025622,0.069365,0.007849,0.005094,...,-0.000715,-0.051582,0.038602,-0.037859,-0.019896,-0.024527,0.005044,0.009022,-0.067295,-0.004310
9,14,2,79,8,3299,-0.059145,-0.034740,0.044045,-0.018557,0.023751,...,0.036723,-0.014096,-0.059855,0.011106,-0.013098,0.064376,-0.001612,-0.070184,-0.029203,-0.014352


In [19]:
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_auc_score
from scipy.stats import mode
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np
from sklearn.neural_network import MLPClassifier

# Separate features (X) and target variable (y)
X = data_ready.drop(drop_list, axis=1) 
y = data_ready['Disposition_recode_2']

# Encode target variable if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)

X_test=data_ready_test.drop(drop_list_test, axis=1)
y_test=data_ready_test['Disposition_recode_2']
y_test = le.fit_transform(y_test)

# Create a scaler for numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['number']))

with open('X_scaler_legacy.pkl', 'wb') as file:
    pickle.dump(scaler, file)

X_test_scaled = scaler.transform(X_test.select_dtypes(include=['number']))

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.select_dtypes(include=['number']).columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.select_dtypes(include=['number']).columns)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled_df, y_train)

# Combine scaled numerical features with categorical features
X_train_final = X_resampled.copy()
X_test_final = X_test_scaled_df.copy()
y_train_final=y_resampled.copy()


# Define models and their parameter grids for hyperparameter tuning
models = {
    #'KNN': (KNeighborsClassifier(), {'knn__n_neighbors': [3, 5, 7]}),
    #'Random_Forest': (RandomForestClassifier(), {'random_forest__n_estimators': [100, 200], 'random_forest__max_depth': [4, 8]}),
    'XGBoost': (XGBClassifier(
                    objective='binary:logistic',
                    # device = "cuda"
                ), 
           {'xgboost__learning_rate': [0.1, 0.01, 0.001],            # Step size shrinkage to prevent overfitting
            'xgboost__max_depth': [3, 5, 7],                         # Maximum depth of each tree
            #'xgboost__n_estimators': [100, 200, 300],                # Number of boosting rounds
            #'xgboost__min_child_weight': [1, 3, 5],                  # Minimum sum of instance weight needed in a child
            #'xgboost__subsample': [0.6, 0.8,0.9],                   # Subsample ratio of the training instances
            #'xgboost__colsample_bytree': [0.6, 0.8, 0.9],            # Subsample ratio of columns when constructing each tree
            #'xgboost__gamma': [0.1, 0.5, 1,3],                     # Minimum loss reduction required for a split
            #'xgboost__reg_alpha': [0.01, 0.1,0.5,1],                    # L1 regularization term on weights
            #'xgboost__reg_lambda': [1, 1.5, 2],                      # L2 regularization term on weights
            'xgboost__scale_pos_weight': [1, 5, 10]
            }),
    #'Logistic_Regression': (LogisticRegression(solver='liblinear'), {'logistic_regression__C': [0.1, 1, 10]}),
    #'SVM': (SVC(probability=True), {'svm__C': [0.1, 1, 10], 'svm__kernel': ['linear', 'rbf']}),
    #'NN': (MLPClassifier(max_iter=1000), {'nn__hidden_layer_sizes': [(10,), (50,), (100,)], 'nn__activation': ['relu', 'tanh']}),
}

# Initialize list to store predictions from each model

ensemble_predictions = []
ensemble_probabilities = []

for model_name, (model, param_grid) in models.items():
    print(f"Training {model_name}...")

    # Create a pipeline for preprocessing and model training
    pipeline = Pipeline([
        (model_name.lower(), model)
    ])

    # Perform hyperparameter tuning using GridSearchCV with AUC score
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='precision',n_jobs=8)
    grid_search.fit(X_train_final, y_train_final)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions and predict probabilities on the test set
    y_pred = best_model.predict(X_test_final)
    y_prob = best_model.predict_proba(X_test_final)[:, 1]  # Probability for the positive class
    ensemble_predictions.append(y_pred)
    ensemble_probabilities.append(y_prob)

    # Evaluate the model using accuracy, F1-score, and AUC
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} F1 Score: {f1}")
    print(f"{model_name} AUC: {auc}")
    print(classification_report(y_test, y_pred))

# Implement ensemble method (majority voting for predictions, average for probabilities)
ensemble_predictions = np.array(ensemble_predictions)
ensemble_pred_final = mode(ensemble_predictions, axis=0)[0].flatten()

# Average the probabilities for the ensemble AUC
ensemble_probabilities = np.mean(ensemble_probabilities, axis=0)

# Evaluate the ensemble model using accuracy, F1-score, and AUC
ensemble_accuracy = accuracy_score(y_test, ensemble_pred_final)
ensemble_f1 = f1_score(y_test, ensemble_pred_final)
ensemble_auc = roc_auc_score(y_test, ensemble_probabilities)
print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble F1 Score:", ensemble_f1)
print("Ensemble AUC:", ensemble_auc)
print(classification_report(y_test, ensemble_pred_final))

with open('xgboost_tfidf_LLM_legacy.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Training XGBoost...
XGBoost Accuracy: 0.8898095238095238
XGBoost F1 Score: 0.44588122605363983
XGBoost AUC: 0.8481501882320907
              precision    recall  f1-score   support

           0       0.94      0.94      0.94     18966
           1       0.43      0.46      0.45      2034

    accuracy                           0.89     21000
   macro avg       0.69      0.70      0.69     21000
weighted avg       0.89      0.89      0.89     21000

Ensemble Accuracy: 0.8898095238095238
Ensemble F1 Score: 0.44588122605363983
Ensemble AUC: 0.8481501882320907
              precision    recall  f1-score   support

           0       0.94      0.94      0.94     18966
           1       0.43      0.46      0.45      2034

    accuracy                           0.89     21000
   macro avg       0.69      0.70      0.69     21000
weighted avg       0.89      0.89      0.89     21000



In [16]:
X_test_final.head(5)


,Age,Sex,Body_Part,Location,Product_1,0,1,2,3,4,...,374,375,376,377,378,379,380,381,382,383
0,0.562805,-0.926837,-1.352263,-0.758244,-0.224021,0.812341,0.019864,0.272011,0.138040,0.071870,...,-1.378340,0.850912,0.391880,-1.092181,0.233195,-1.411935,1.094539,-0.671935,0.145551,0.281539
1,-0.631310,1.078938,0.452933,-0.758244,-0.192975,-1.028592,-1.113329,-0.943803,0.246058,-0.834536,...,-0.674765,0.805552,0.301963,-0.690093,-0.388876,-0.495041,-0.488629,-1.464057,-0.049527,-0.936497
2,-0.589411,-0.926837,-1.438224,1.728633,2.142138,1.460908,-0.139787,-0.975974,-1.404326,0.223596,...,0.650409,0.209710,-0.125878,1.197081,0.400591,-0.315888,-0.188688,-1.110939,2.615629,-0.021017
3,-0.086626,-0.926837,-1.438224,-0.447385,-0.240284,0.024341,-1.192843,0.344265,-2.604271,1.690395,...,-0.394167,-1.166810,2.127170,1.702479,0.577233,-0.393100,-0.436046,-0.052158,0.130527,-0.410507
4,-0.777956,-0.926837,-1.352263,0.485195,0.852984,-0.912231,-0.210948,0.481260,1.416604,-1.190152,...,1.614138,0.206727,0.408004,-0.081440,-0.900475,-0.230047,-0.310264,0.327073,-1.297561,-0.891554


In [17]:
scaled_body_parts = X_test_final['Body_Part'].unique()
scaled_body_parts.sort()
scaled_body_parts

array([-1.4812053 , -1.43822444, -1.39524359, -1.35226273, -1.30928187,
       -1.26630102, -1.22332016, -1.18033931, -1.13735845,  0.4529332 ,
        0.49591405,  0.53889491,  0.62485662,  0.66783748,  0.71081833,
        0.75379919,  0.79678004,  1.01168432,  1.05466518,  1.18360774,
        1.2265886 ,  1.26956945])

In [18]:
for index, row in X_test_final[:10].iterrows():
    prob = []
    for body_part in scaled_body_parts:
        current_sample = row.copy()
        current_sample['Body_Part'] = body_part
        # print(current_sample)
        prob.append(best_model.predict_proba([current_sample])[0][0])
    prob.sort()
    print(prob[0], prob[-1]) # min max

0.29972845 0.8825259
0.79223377 0.99258816
0.67965525 0.9739398
0.79754674 0.98187
0.5333736 0.9794353
0.59031236 0.9827104
0.6931846 0.9906752
0.4409511 0.8889836
0.38819468 0.93166035
0.56050295 0.9667125
